In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
path_2022 = "../data/movilidad_provincias_2022.csv"
path_2023 = "../data/movilidad_provincias_2023.csv"
path_2024 = "../data/movilidad_provincias_2024.csv"

original_data_2022 = pd.read_csv(path_2022, sep=",")
original_data_2023 = pd.read_csv(path_2023, sep=",")
original_data_2024 = pd.read_csv(path_2024, sep=",")

df_2022 = original_data_2022.copy()
df_2023 = original_data_2023.copy()
df_2024 = original_data_2024.copy()

og_df = pd.concat([df_2022, df_2023, df_2024])
df = og_df.copy()

In [3]:
path_festivos_2022 = "../data/bank_holidays_22.csv"
path_festivos_2023_24 = "../data/bank_holidays_2324.csv"

festivos_2022 = pd.read_csv(path_festivos_2022, sep=";", encoding='ISO-8859-1')
festivos_2023_24 = pd.read_csv(path_festivos_2023_24, sep=";", encoding='ISO-8859-1')
festivos = pd.concat([festivos_2022, festivos_2023_24])

#festivos['holiday_date'] = pd.to_datetime(festivos['holiday_date'])

In [15]:
# Juntar los datasets de movilidad con los de festivos por day = holiday_date y provincia_destino_name = desc_provincia (dejando en null las filas que no tengan festivo)
# 1. Hacer el merge inicial
merged_df = pd.merge(
    df, 
    festivos, 
    left_on=['day', 'provincia_destino_name'], 
    right_on=['holiday_date', 'desc_provincia'], 
    how='left'
)

# 2. Crear columnas dummy y renombrarlas
merged_df = pd.get_dummies(merged_df, columns=['holiday_type'])
merged_df = merged_df.rename(columns={
    'holiday_type_Nacional': 'national_holiday', 
    'holiday_type_Autonómico': 'regional_holiday'
})

# 3. Convertir a int manteniendo todas las columnas
merged_df['national_holiday'] = merged_df['national_holiday'].fillna(0).astype(int)
merged_df['regional_holiday'] = merged_df['regional_holiday'].fillna(0).astype(int)

# 4. Ver resultados con festivos
merged_df_holiday = merged_df[merged_df['holiday_date'].notnull()]
merged_df_holiday

,viajeros,viajes,provincia_origen,provincia_origen_name,provincia_destino,provincia_destino_name,day,cod_provincia,desc_provincia,ine_auto_name,holiday_date,holiday_desc,regional_holiday,national_holiday
2469,4666,5468,11,Cádiz,51,Ceuta,2022-09-02,51.0,Ceuta,"Ceuta, Ciudad Autónoma de",2022-09-02,Día de Ceuta,1,0
2605,38,38,18,Granada,51,Ceuta,2022-09-02,51.0,Ceuta,"Ceuta, Ciudad Autónoma de",2022-09-02,Día de Ceuta,1,0
2958,478,483,29,Málaga,51,Ceuta,2022-09-02,51.0,Ceuta,"Ceuta, Ciudad Autónoma de",2022-09-02,Día de Ceuta,1,0
3312,22,24,41,Sevilla,51,Ceuta,2022-09-02,51.0,Ceuta,"Ceuta, Ciudad Autónoma de",2022-09-02,Día de Ceuta,1,0
12755,55,55,2,Albacete,6,Badajoz,2022-09-08,6.0,Badajoz,Extremadura,2022-09-08,Día de Extremadura,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1312990,5368,5490,50,Zaragoza,8,Barcelona,2024-09-11,8.0,Barcelona,Cataluña,2024-09-11,Fiesta Nacional de Cataluña,1,0
1313001,430,471,50,Zaragoza,17,Girona,2024-09-11,17.0,Girona,Cataluña,2024-09-11,Fiesta Nacional de Cataluña,1,0
1313008,6341,6406,50,Zaragoza,25,Lleida,2024-09-11,25.0,Lleida,Cataluña,2024-09-11,Fiesta Nacional de Cataluña,1,0
1313023,3082,3174,50,Zaragoza,43,Tarragona,2024-09-11,43.0,Tarragona,Cataluña,2024-09-11,Fiesta Nacional de Cataluña,1,0


In [73]:
# Añadir columna de vacaciones verano
fechas_verano = (
    pd.concat([
        # Verano 2023
        pd.Series(pd.date_range(start='2023-07-01', end='2023-08-31')),
        # Verano 2024
        pd.Series(pd.date_range(start='2024-07-01', end='2024-08-31'))
    ])
    .dt.strftime('%Y-%m-%d')
    .tolist()
)

# Crear una columna en el dataset que indique si es verano o no
merged_df['es_verano'] = merged_df['day'].isin(fechas_verano)
merged_df['es_verano'] = merged_df['es_verano'].astype(int)

In [74]:
# Añadir columna de vacaciones navidad
fechas_navidad = (
    pd.concat([
        # Navidad 2022-2023
        pd.Series(pd.date_range(start='2022-12-22', end='2023-01-08')),
        # Navidad 2023-2024
        pd.Series(pd.date_range(start='2023-12-22', end='2024-01-07'))
    ])
    .dt.strftime('%Y-%m-%d')
    .tolist()
)

# Crear una columna en el dataset que indique si es navidad o no
merged_df['es_navidad'] = merged_df['day'].isin(fechas_navidad)
merged_df['es_navidad'] = merged_df['es_navidad'].astype(int)

In [76]:
# Añadir columna de vacaciones semana santa
fechas_semana_santa = (
    pd.concat([
        # Semana santa 2023
        pd.Series(pd.date_range(start='2023-04-02', end='2023-04-09')),
        # Semana santa 2024
        pd.Series(pd.date_range(start='2024-03-24', end='2024-03-31'))
    ])
    .dt.strftime('%Y-%m-%d')
    .tolist()
)

# Crear una columna en el dataset que indique si es semana santa o no
merged_df['es_semana_santa'] = merged_df['day'].isin(fechas_semana_santa)
merged_df['es_semana_santa'] = merged_df['es_semana_santa'].astype(int)

In [ ]:
# Añadir columna de dia marcado

In [41]:
# Mostrar solo las filas conicidentes con festivos
""" merged_df_filtered = merged_df[merged_df['holiday_date'].notnull()]
merged_df_filtered = merged_df_filtered[merged_df_filtered['holiday_date'] == '2024-09-11']
merged_df_filtered = merged_df_filtered[merged_df_filtered['provincia_destino_name'] == 'Barcelona']
merged_df_filtered """

" merged_df_filtered = merged_df[merged_df['holiday_date'].notnull()]\nmerged_df_filtered = merged_df_filtered[merged_df_filtered['holiday_date'] == '2024-09-11']\nmerged_df_filtered = merged_df_filtered[merged_df_filtered['provincia_destino_name'] == 'Barcelona']\nmerged_df_filtered "

In [42]:
# Filter by province
""" insular_provinces = ['Balears, Illes', 'Palmas, Las', 'Santa Cruz de Tenerife']
df = df[df['provincia_destino_name'].isin(insular_provinces)]
df """

" insular_provinces = ['Balears, Illes', 'Palmas, Las', 'Santa Cruz de Tenerife']\ndf = df[df['provincia_destino_name'].isin(insular_provinces)]\ndf "